In [3]:
from kaggle_environments import make
import json, math, random, os
from itertools import combinations
import pandas as pd

Loading environment football failed: No module named 'gfootball'


In [10]:
def get_results(step):
    city_tiles = [0, 0]
    units = [0, 0]
    for update in step[0]['observation']['updates']:
        split = update.split(' ')
        if split[0] == 'u':
            units[int(split[2])] += 1
        elif split[0] == 'ct':
            city_tiles[int(split[1])] += 1
            
    if city_tiles[0] > city_tiles[1]:
        winner = 0
    elif city_tiles[0] < city_tiles[1]:
        winner = 1
    elif units[0] > units[1]:
        winner = 0
    elif units[0] < units[1]:
        winner = 1
    else:
        winner = -1 # tie
        
    return(city_tiles[0], units[0], city_tiles[1], units[1], winner)


def play_seed(seed, debug=False):
    env = make('lux_ai_2021', configuration={'seed': seed, 'loglevel': 2, 'annotations': False}, debug=True)
    match_results = []
    chosen_agents = []
    cols = ['seed', 'bot0', 'city_tiles0', 'units0', 'bot1', 'city_tiles1', 'units1', 'winner']
    bots = ['agent.py', './working_title/agent.py', './kaban/agent_tb.py', './kaban/agent_dr.py', './kaban/agent_rl.py']
    names = ['meta_bot', 'working_title', 'imitation_tb', 'imitation_dr', 'imitation_rl']
    
    algo = None
    
    df_ca_list = []
    # when meta_bot playing, need to determine most played arm
    for (bot0, name0), (bot1, name1) in combinations(zip(bots, names), 2):
        # play match
        print("****************************************")
        print(f"Playing {name0} vs {name1}, seed {seed}")
        steps = env.run([bot0, bot1])
        city_tiles0, units0, city_tiles1, units1, winner = get_results(steps[-1])
        
        if winner == 0:
            winner_name = name0
        elif winner == 1:
            winner_name = name1
        else:
            winner_name = 'tie'        
        
        print("****************************************")
        print(f"In {name0} vs {name1}, {winner_name} wins! {name0} had {city_tiles0} city_tiles and {units0} units while {name1} had {city_tiles1} city_tiles and {units1} units.")
        match_results.append([seed, name0, city_tiles0, units0, name1, city_tiles1, units1, winner_name])
        
        
        if 'meta_bot' in [name0, name1]:
            # stores which agent was chosen at each time step by the OL algo
            df_ca_single = pd.read_csv('chosen_agent.log', names=['agent', 'raw_reward', 'scaled_reward'], skiprows=2)
            df_ca_single.insert(0, 'seed', seed)
            df_ca_single.insert(1, 'step', range(df_ca_single.shape[0]))
            df_ca_list.append(df_ca_single)
        
        # get name of OL algo being used
        if algo is None:
            with open('chosen_agent.log', 'r') as f:
                algo = f.readline().rstrip()
        
    df_mr = pd.DataFrame(match_results, columns=cols)
    df_mr.insert(0, 'algo', algo)
    df_ca = pd.concat(df_ca_list)
    df_ca.insert(0, 'algo', algo)
    return(df_mr, df_ca)
    
seeds_played = []
def experiments(n, debug=False):
    global seeds_played
    
    chosen_agents_list = []
    results_list = []
    
    for i in range(n):
        
        seed = math.floor(random.random() * 1e9)
        while seed in seeds_played:
            seed = math.floor(random.random() * 1e9)
        
        results, chosen_agents = play_seed(seed, debug)
        seeds_played.append(seed)
        results_list.append(results)
        chosen_agents_list.append(chosen_agents)
        
    df_results = pd.concat(results_list)
    df_chosen_agents = pd.concat(chosen_agents_list)
        
    # save results
    os.makedirs('seed_experiments', exist_ok=True)
    if 'results.csv' in os.listdir('seed_experiments'):
        df_results_orig = pd.read_csv(os.path.join('seed_experiments', 'results.csv'))
        df_results = pd.concat([df_results_orig, df_results])
        
    if 'chosen_agents.csv' in os.lsitdir('seed_experiments'):
        df_chosen_agents_orig = pd.read_csv(os.path.join('seed_experiments', 'chosen_agents.csv'))
        df_chosen_agents = pd.concat([df_chosen_agents_orig, df_chosen_agents])

    df_results.to_csv(os.path.join('seed_experiments', 'results.csv'), index=False)
    df_chosen_agents.to_csv(os.path.join('seed_experiments', 'chosen_agents.csv'), index=False)

In [11]:
results,chosen_agents = play_seed(1)
print(results)
print(chosen_agents)

****************************************
Playing meta_bot vs working_title, seed 1
agent.py:106: RuntimeWarning: invalid value encountered in double_scalars
  return(self.c * np.log(self.t ** 2)/(self.t * delta_hat ** 2))
agent.py:106: RuntimeWarning: divide by zero encountered in double_scalars
  return(self.c * np.log(self.t ** 2)/(self.t * delta_hat ** 2))
rew/r_total=0.30,rew/r_wood=0.00,rew/r_coal=0.00,rew/r_uranium=0.00,rew/r_research=0.00,rew/r_city_tiles_end=0.00,rew/r_fuel_collected=0.00,rew/r_units=0.10,rew/r_city_tiles=0.20,game/turns=0.00,game/research=1.00,game/unit_count=2.00,game/cart_count=0.00,game/city_count=2.00,game/city_tiles=2.00,game/wood_rate_mined=0.00,game/coal_rate_mined=0.00,game/uranium_rate_mined=0.00,rew/r_carts=0.00
rew/r_total=0.61,rew/r_wood=0.00,rew/r_coal=0.00,rew/r_uranium=0.00,rew/r_research=0.01,rew/r_city_tiles_end=0.00,rew/r_fuel_collected=0.00,rew/r_units=0.20,rew/r_city_tiles=0.40,game/turns=0.00,game/research=3.00,game/unit_count=4.00,game/ca

[WARN] (match_oizjV0wQnQyJ) - Agent 1 tried to move unit u_11 onto opponent CityTile; turn 37; cmd: m u_11 w
[WARN] (match_oizjV0wQnQyJ) - Agent 1 tried to build CityTile with insufficient materials wood + coal + uranium: 96; turn 37; cmd: bcity u_4
[WARN] (match_oizjV0wQnQyJ) - Agent 1 tried to build CityTile with insufficient materials wood + coal + uranium: 96; turn 37; cmd: bcity u_9
[WARN] (match_oizjV0wQnQyJ) - Agent 1 tried to move unit u_11 onto opponent CityTile; turn 38; cmd: m u_11 w
[WARN] (match_oizjV0wQnQyJ) - Agent 1 tried to build CityTile with insufficient materials wood + coal + uranium: 96; turn 38; cmd: bcity u_10
[WARN] (match_oizjV0wQnQyJ) - Agent 1 tried to build CityTile with insufficient materials wood + coal + uranium: 96; turn 38; cmd: bcity u_14
[WARN] (match_oizjV0wQnQyJ) - Agent 1 tried to build CityTile with insufficient materials wood + coal + uranium: 96; turn 38; cmd: bcity u_4
[WARN] (match_oizjV0wQnQyJ) - Agent 1 tried to build CityTile with insuffic

[WARN] (match_oizjV0wQnQyJ) - Agent 0 tried to build CityTile on non-empty resource tile; turn 60; cmd: bcity u_18
[WARN] (match_oizjV0wQnQyJ) - Agent 1 tried to move unit u_27 onto opponent CityTile; turn 60; cmd: m u_27 e
[WARN] (match_oizjV0wQnQyJ) - turn 60; Unit u_12 collided when trying to move s to (5, 15)
[WARN] (match_oizjV0wQnQyJ) - turn 64; Unit u_18 collided when trying to move w to (11, 13)
[WARN] (match_oizjV0wQnQyJ) - turn 64; Unit u_20 collided when trying to move s to (12, 13)
[WARN] (match_oizjV0wQnQyJ) - turn 65; Unit u_6 collided when trying to move e to (11, 4)
[WARN] (match_oizjV0wQnQyJ) - turn 66; Unit u_18 collided when trying to move w to (11, 13)
[WARN] (match_oizjV0wQnQyJ) - Agent 1 tried to move unit u_10 onto opponent CityTile; turn 67; cmd: m u_10 n
[WARN] (match_oizjV0wQnQyJ) - turn 67; Unit u_18 collided when trying to move w to (11, 13)
[WARN] (match_oizjV0wQnQyJ) - turn 68; Unit u_18 collided when trying to move w to (11, 13)
[WARN] (match_oizjV0wQnQyJ

[WARN] (match_oizjV0wQnQyJ) - turn 90; Unit u_45 collided when trying to move w to (11, 4)
[WARN] (match_oizjV0wQnQyJ) - turn 90; Unit u_44 collided when trying to move e to (11, 4)
[WARN] (match_oizjV0wQnQyJ) - turn 91; Unit u_45 collided when trying to move w to (11, 4)
[WARN] (match_oizjV0wQnQyJ) - turn 91; Unit u_44 collided when trying to move e to (11, 4)
[WARN] (match_oizjV0wQnQyJ) - turn 91; Unit u_46 collided when trying to move s to (11, 3)
rew/r_total=4.25,rew/r_wood=0.00,rew/r_coal=0.00,rew/r_uranium=0.00,rew/r_research=0.15,rew/r_city_tiles_end=0.00,rew/r_fuel_collected=0.00,rew/r_units=1.40,rew/r_city_tiles=2.70,game/turns=0.00,game/research=59.00,game/unit_count=28.00,game/cart_count=0.00,game/city_count=17.00,game/city_tiles=27.00,game/wood_rate_mined=0.00,game/coal_rate_mined=0.00,game/uranium_rate_mined=0.00,rew/r_carts=0.00
[WARN] (match_oizjV0wQnQyJ) - turn 92; Unit u_45 collided when trying to move w to (11, 4)
[WARN] (match_oizjV0wQnQyJ) - turn 92; Unit u_44 colli

[WARN] (match_oizjV0wQnQyJ) - Agent 1 tried to move unit u_27 onto opponent CityTile; turn 104; cmd: m u_27 n
[WARN] (match_oizjV0wQnQyJ) - turn 104; Unit u_18 collided when trying to move n to (12, 9)
[WARN] (match_oizjV0wQnQyJ) - turn 104; Unit u_21 collided when trying to move w to (12, 10)
[WARN] (match_oizjV0wQnQyJ) - turn 104; Unit u_9 collided when trying to move s to (12, 9)
[WARN] (match_oizjV0wQnQyJ) - turn 104; Unit u_36 collided when trying to move s to (11, 9)
[WARN] (match_oizjV0wQnQyJ) - turn 104; Unit u_22 collided when trying to move e to (11, 8)
[WARN] (match_oizjV0wQnQyJ) - turn 104; Unit u_31 collided when trying to move e to (11, 9)
[WARN] (match_oizjV0wQnQyJ) - turn 104; Unit u_45 collided when trying to move w to (11, 4)
[WARN] (match_oizjV0wQnQyJ) - turn 104; Unit u_44 collided when trying to move e to (11, 4)
[WARN] (match_oizjV0wQnQyJ) - turn 104; Unit u_48 collided when trying to move s to (4, 8)
[WARN] (match_oizjV0wQnQyJ) - turn 104; Unit u_50 collided when

...

[WARN] (match_bklqkXWl5iZ7) - Agent 0 tried to build CityTile on non-empty resource tile; turn 179; cmd: bcity u_1
[WARN] (match_bklqkXWl5iZ7) - Agent 1 tried to build CityTile with insufficient materials wood + coal + uranium: 84; turn 179; cmd: bcity u_38
[WARN] (match_bklqkXWl5iZ7) - Agent 1 tried to move unit u_69 off map; turn 179; cmd: m u_69 e
[WARN] (match_bklqkXWl5iZ7) - turn 179; Unit u_40 collided when trying to move e to (15, 2)
[WARN] (match_bklqkXWl5iZ7) - turn 179; Unit u_31 collided when trying to move n to (15, 2)
[WARN] (match_bklqkXWl5iZ7) - turn 179; Unit u_14 collided when trying to move n to (15, 3)
[WARN] (match_bklqkXWl5iZ7) - turn 179; Unit u_42 collided when trying to move n to (15, 4)
[WARN] (match_bklqkXWl5iZ7) - turn 179; Unit u_80 collided when trying to move s to (5, 15)
[WARN] (match_bklqkXWl5iZ7) - turn 179; Unit u_17 collided when trying to move w to (5, 15)
[WARN] (match_bklqkXWl5iZ7) - turn 179; Unit u_76 collided when trying to move s to (15, 15)
[W

[WARN] (match_bklqkXWl5iZ7) - Agent 1 tried to move unit u_15 onto opponent CityTile; turn 189; cmd: m u_15 n
[WARN] (match_bklqkXWl5iZ7) - Agent 1 tried to move unit u_69 off map; turn 189; cmd: m u_69 e
[WARN] (match_bklqkXWl5iZ7) - turn 189; Unit u_80 collided when trying to move s to (5, 15)
[WARN] (match_bklqkXWl5iZ7) - turn 189; Unit u_17 collided when trying to move w to (5, 15)
[WARN] (match_bklqkXWl5iZ7) - turn 189; Unit u_63 collided when trying to move e to (15, 15)
[WARN] (match_bklqkXWl5iZ7) - Agent 1 tried to move unit u_15 onto opponent CityTile; turn 190; cmd: m u_15 n
[WARN] (match_bklqkXWl5iZ7) - Agent 1 tried to move unit u_69 off map; turn 190; cmd: m u_69 e
[WARN] (match_bklqkXWl5iZ7) - turn 190; Unit u_80 collided when trying to move s to (5, 15)
[WARN] (match_bklqkXWl5iZ7) - turn 190; Unit u_17 collided when trying to move w to (5, 15)
[WARN] (match_bklqkXWl5iZ7) - turn 190; Unit u_63 collided when trying to move e to (15, 15)
[WARN] (match_bklqkXWl5iZ7) - Agent 

[WARN] (match_bklqkXWl5iZ7) - turn 218; Unit u_1 collided when trying to move n to (11, 5)
[WARN] (match_bklqkXWl5iZ7) - turn 218; Unit u_15 collided when trying to move w to (11, 5)
[WARN] (match_bklqkXWl5iZ7) - turn 219; Unit u_1 collided when trying to move n to (11, 5)
[WARN] (match_bklqkXWl5iZ7) - turn 219; Unit u_15 collided when trying to move w to (11, 5)
[WARN] (match_bklqkXWl5iZ7) - turn 220; Unit u_1 collided when trying to move n to (11, 5)
[WARN] (match_bklqkXWl5iZ7) - turn 220; Unit u_15 collided when trying to move w to (11, 5)
[WARN] (match_bklqkXWl5iZ7) - turn 221; Unit u_1 collided when trying to move n to (11, 5)
[WARN] (match_bklqkXWl5iZ7) - turn 221; Unit u_15 collided when trying to move w to (11, 5)
[WARN] (match_bklqkXWl5iZ7) - turn 222; Unit u_1 collided when trying to move n to (11, 5)
[WARN] (match_bklqkXWl5iZ7) - turn 222; Unit u_15 collided when trying to move w to (11, 5)
[WARN] (match_bklqkXWl5iZ7) - Agent 1 tried to move unit u_15 onto opponent CityTile;

[WARN] (match_bklqkXWl5iZ7) - Agent 0 tried to build CityTile with insufficient materials wood + coal + uranium: 92; turn 268; cmd: bcity u_60
[WARN] (match_bklqkXWl5iZ7) - Agent 1 tried to move unit u_15 onto opponent CityTile; turn 268; cmd: m u_15 n
[WARN] (match_bklqkXWl5iZ7) - Agent 0 tried to build CityTile with insufficient materials wood + coal + uranium: 92; turn 269; cmd: bcity u_60
[WARN] (match_bklqkXWl5iZ7) - Agent 1 tried to move unit u_15 onto opponent CityTile; turn 269; cmd: m u_15 n
[WARN] (match_bklqkXWl5iZ7) - Agent 0 tried to build CityTile with insufficient materials wood + coal + uranium: 92; turn 270; cmd: bcity u_60
[WARN] (match_bklqkXWl5iZ7) - Agent 1 tried to move unit u_15 onto opponent CityTile; turn 270; cmd: m u_15 n
[WARN] (match_bklqkXWl5iZ7) - Agent 0 tried to build CityTile with insufficient materials wood + coal + uranium: 88; turn 271; cmd: bcity u_60
[WARN] (match_bklqkXWl5iZ7) - Agent 1 tried to move unit u_15 onto opponent CityTile; turn 271; cm

[WARN] (match_bklqkXWl5iZ7) - Agent 0 tried to build CityTile with insufficient materials wood + coal + uranium: 80; turn 315; cmd: bcity u_92
[WARN] (match_bklqkXWl5iZ7) - Agent 0 tried to build CityTile with insufficient materials wood + coal + uranium: 80; turn 315; cmd: bcity u_139
[WARN] (match_bklqkXWl5iZ7) - Agent 1 tried to move unit u_15 onto opponent CityTile; turn 315; cmd: m u_15 n
[WARN] (match_bklqkXWl5iZ7) - Agent 1 tried to move unit u_15 onto opponent CityTile; turn 316; cmd: m u_15 n
[WARN] (match_bklqkXWl5iZ7) - Agent 1 tried to move unit u_15 onto opponent CityTile; turn 317; cmd: m u_15 n
[WARN] (match_bklqkXWl5iZ7) - Agent 1 tried to move unit u_15 onto opponent CityTile; turn 318; cmd: m u_15 n
[WARN] (match_bklqkXWl5iZ7) - Agent 1 tried to move unit u_15 onto opponent CityTile; turn 319; cmd: m u_15 n
[WARN] (match_bklqkXWl5iZ7) - Agent 1 tried to move unit u_15 onto opponent CityTile; turn 324; cmd: m u_15 n
[WARN] (match_bklqkXWl5iZ7) - Agent 1 tried to move u

In [12]:
results

,algo,seed,bot0,city_tiles0,units0,bot1,city_tiles1,units1,winner
0,EXP3PP,1,meta_bot,0,0,working_title,6,8,1
1,EXP3PP,1,meta_bot,3,3,imitation_toad_brigade,2,4,0
2,EXP3PP,1,meta_bot,1,4,imitation_dr,4,6,1
3,EXP3PP,1,meta_bot,6,13,imitation_rl,1,2,0
4,EXP3PP,1,working_title,0,0,imitation_toad_brigade,18,13,1
5,EXP3PP,1,working_title,0,0,imitation_dr,34,34,1
6,EXP3PP,1,working_title,1,3,imitation_rl,0,4,0
7,EXP3PP,1,imitation_toad_brigade,27,26,imitation_dr,0,0,0
8,EXP3PP,1,imitation_toad_brigade,14,14,imitation_rl,0,1,0
9,EXP3PP,1,imitation_dr,29,29,imitation_rl,0,2,0


In [13]:
chosen_agents

,algo,seed,step,agent,raw_reward,scaled_reward
0,EXP3PP,1,0,imitation_rl,0.1525,0.012200
1,EXP3PP,1,1,imitation_toad_brigade,0.0000,0.000000
2,EXP3PP,1,2,imitation_toad_brigade,0.0000,0.000000
3,EXP3PP,1,3,imitation_toad_brigade,0.0000,0.000000
4,EXP3PP,1,4,imitation_rl,0.1000,0.007899
...,...,...,...,...,...,...
354,EXP3PP,1,354,imitation_toad_brigade,0.0000,0.000000
355,EXP3PP,1,355,imitation_toad_brigade,0.0000,0.000000
356,EXP3PP,1,356,working_title,0.0000,0.000000
357,EXP3PP,1,357,imitation_dr,0.0000,0.000000


In [6]:
env = make('lux_ai_2021', configuration={'seed': 1, 'loglevel': 2, 'annotations': False}, debug=True)
env.run(['agent.py', './working_title/agent.py'])

agent.py:106: RuntimeWarning: invalid value encountered in double_scalars
  return(self.c * np.log(self.t ** 2)/(self.t * delta_hat ** 2))
agent.py:106: RuntimeWarning: divide by zero encountered in double_scalars
  return(self.c * np.log(self.t ** 2)/(self.t * delta_hat ** 2))
rew/r_total=0.15,rew/r_wood=0.00,rew/r_coal=0.00,rew/r_uranium=0.00,rew/r_research=0.00,rew/r_city_tiles_end=0.00,rew/r_fuel_collected=0.00,rew/r_units=0.05,rew/r_city_tiles=0.10,game/turns=0.00,game/research=1.00,game/unit_count=1.00,game/cart_count=0.00,game/city_count=1.00,game/city_tiles=1.00,game/wood_rate_mined=0.00,game/coal_rate_mined=0.00,game/uranium_rate_mined=0.00,rew/r_carts=0.00
rew/r_total=0.15,rew/r_wood=0.00,rew/r_coal=0.00,rew/r_uranium=0.00,rew/r_research=0.00,rew/r_city_tiles_end=0.00,rew/r_fuel_collected=0.00,rew/r_units=0.05,rew/r_city_tiles=0.10,game/turns=0.00,game/research=1.00,game/unit_count=1.00,game/cart_count=0.00,game/city_count=1.00,game/city_tiles=1.00,game/wood_rate_mined=0.00,g

[WARN] (match_Um8Zok1GL1Um) - Agent 1 tried to build CityTile with insufficient materials wood + coal + uranium: 96; turn 37; cmd: bcity u_2
[WARN] (match_Um8Zok1GL1Um) - Agent 1 tried to build CityTile with insufficient materials wood + coal + uranium: 96; turn 37; cmd: bcity u_14
[WARN] (match_Um8Zok1GL1Um) - Agent 1 tried to build CityTile with insufficient materials wood + coal + uranium: 96; turn 37; cmd: bcity u_9
[WARN] (match_Um8Zok1GL1Um) - Agent 0 tried to build CityTile with insufficient materials wood + coal + uranium: 96; turn 38; cmd: bcity u_6
[WARN] (match_Um8Zok1GL1Um) - Agent 1 tried to build CityTile with insufficient materials wood + coal + uranium: 96; turn 38; cmd: bcity u_2
[WARN] (match_Um8Zok1GL1Um) - Agent 1 tried to build CityTile with insufficient materials wood + coal + uranium: 96; turn 38; cmd: bcity u_14
[WARN] (match_Um8Zok1GL1Um) - Agent 1 tried to build CityTile with insufficient materials wood + coal + uranium: 96; turn 38; cmd: bcity u_9
[WARN] (mat

KeyboardInterrupt: 